In [4]:
import dns.name
import dns.query
import dns.dnssec
import dns.message
import dns.resolver
import dns.rdatatype
import re
import ipaddress
import time
import datetime
import random

In [5]:
rdtype_dic = {
    'A': 1      ,
    'NS':2      ,
    'DS':43     ,
    'RRSIG': 46 ,
    'DNSKEY':48 ,
}

In [6]:
def print_response(response):
    '''Print the response's answer, authority, and additional
    
    Args:
        response (dns.message.Message): a response of a single iterative DNS query
    '''
    print('\n', 'QUESTION:')
    for i in response.question:
        print(i.to_text())
    
    print('\n', 'ANSWER:')
    for i in response.answer:
        print(i.to_text())
    
    print('\n', 'AUTHORITY:')
    for i in response.authority:
        print(i.to_text())
        
    print('\n', 'ADDITIONAL:')
    for i in response.additional:
        print(i.to_text())

# Paypal

In [7]:
a_query = dns.message.make_query('paypal.com', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print_response(response1)


 QUESTION:
paypal.com. IN A

 ANSWER:

 AUTHORITY:
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.
com. 86400 IN DS 30909 8 2 e2d3c916f6deeac73294e8268fb5885044a833fc5459588f4a9184cfc41a5766
com. 86400 IN RRSIG DS 8 1 86400 20180304050000 20180219040000 41824 . c4u9828FARDtrKaT70N1cwIKt7GgEPWW /f97LIq68FmvZguNzyIl83trGYPmcieb rJOpPYay5nF4Qi8eupQ01oVNzJxi7Pb9 oI0DO4vYMpUQGsPn2tGYu8ih+C2uoWgY hWecmfVcR9Ch9IfDlafO1mTezdznLQtl BEQJFaJIHBu1IDsXZmQRdRV2px/lWYSz vuOKl65yRNM599hXwON2HRbOIqhJKOGk K20PaxA8sXfQP2xqscONN49I/N5Doem5 ILc9X6diPYiamvuFe8YLtAVpTxW

In [8]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print_response(response2)


 QUESTION:
. IN DNSKEY

 ANSWER:
. 172800 IN DNSKEY 257 3 8 AwEAAagAIKlVZrpC6Ia7gEzahOR+9W29 euxhJhVVLOyQbSEW0O8gcCjFFVQUTf6v 58fLjwBd0YI0EzrAcQqBGCzh/RStIoO8 g0NfnfL2MTJRkxoXbfDaUeVPQuYEhg37 NZWAJQ9VnMVDxP/VHL496M/QZxkjf5/E fucp2gaDX6RS6CXpoY68LsvPVjR0ZSwz z1apAzvN9dlzEheX7ICJBBtuA6G3LQpz W5hOA2hzCTMjJPJ8LbqF6dsV6DoBQzgu l0sGIcGOYl7OyQdXfZ57relSQageu+ip AdTTJ25AsRTAoub8ONGcLmqrAmRLKBP1 dfwhYB4N7knNnulqQxA+Uk1ihz0=
. 172800 IN DNSKEY 257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexT BAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq 7HrxRixHlFlExOLAJr5emLvN7SWXgnLh 4+B5xQlNVz8Og8kvArMtNROxVQuCaSnI DdD5LKyWbRd2n9WGe2R8PzgCmr3EgVLr jyBxWezF0jLHwVN8efS3rCj/EWgvIWgb 9tarpVUDK/b58Da+sqqls3eNbuv7pr+e oZG+SrDK6nWeL3c6H5Apxz7LjVc1uTId sIXxuOLYA4/ilBmSVIzuDWfdRUfhHdY6 +cn8HFRm+2hM8AnXGXws9555KrUB5qih ylGa8subX2Nn6UwNR1AkUTV74bU=
. 172800 IN DNSKEY 256 3 8 AwEAAaDJd0KOMYGCEF0/cftC2hrFtz5G Sn1HOiaxEp053AfbxQ3pT8BEtahPiUkC o1Qx4PECJ23YwaFhfWWjapr6AFxhD8kl fZGp95ickoRlm91ZzXX/mcfn9vlUpZK2 M8qjljNMzZJSopFY+cxRvib2Irb6YeP2 a0

In [9]:
def get_rrset(response, rdtype):
    '''Get the desired rrset (DNSKEY, DS, A, NS), RRSIG and name from the response, their RRSIG
    
    Args:
        response (dns.message.Message): a response of a single iterative DNS query
        rdtype (str): rrset type
        
    Return:
        (rrset, rrsig, name) of desired rdtype
    '''
    try:
        if rdtype == 'DNSKEY' or rdtype == 'A':
            dnskey_or_a, rrsig, name = '', '', ''
            for rrset in response.answer:      # from observation, DNSKEY and A record is in ANSWER section
                if rrset.rdtype == rdtype_dic['RRSIG']:
                    rrsig = rrset
                else:   # rrset.rdtype == rdtype_dic['DNSKEY'] or ['A']:
                    dnskey_or_a = rrset
                    name = rrset.name
            return dnskey_or_a, rrsig, name
        if rdtype == 'DS' or rdtype == 'NS':
            ds_or_ns, rrsig, name = '', '', ''
            for rrset in response.authority:   # from observation, DS and NS record is in AUTHORITY section
                if rrset.rdtype == rdtype_dic['RRSIG']:
                    rrsig = rrset
                else:
                    ds_or_ns = rrset
                    name = rrset.name
            return ds_or_ns, rrsig, name
    except Exception as e:
        print('Oops! Bug in get_rrset')
        raise e          

In [10]:
def verify_dnskey(response):
    '''Verify the dnskey in response. If success, return name_key and dnskey
    
    Args:
        response (dns.message.Message): a response that contains DNSKEY record
        
    Return:
        return (dnskey, name) if success
    '''
    try:
        dnskey, rrsig_key, name_key = get_rrset(response, 'DNSKEY')
        dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
    except Exception as e:
        raise e
    else:
        print('Congrats! DNSKEYs are good~', name_key)
        return name_key, dnskey

In [11]:
verify_dnskey(response2)

Congrats! DNSKEYs are good~ .


(<DNS name .>, <DNS . IN DNSKEY RRset>)

In [12]:
# verify the DNSKEY rrset

try:
    dnskey, rrsig_key, name_key = get_rrset(response2, 'DNSKEY')
    dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DNSKEYs are good~')

Congrats! DNSKEYs are good~


In [13]:
type(dnskey)

dns.rrset.RRset

In [14]:
# verify the DS rrset

try:
    ds, rrsig_ds, name_ds = get_rrset(response1, 'DS')
    dns.dnssec.validate(ds, rrsig_ds, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DS is good~')

Congrats! DS is good~


In [15]:
trust_anchors = [
    # KSK-2017:
    dns.rrset.from_text('.', 1    , 'IN', 'DNSKEY', '257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexTBAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq7HrxRixHlFlExOLAJr5emLvN7SWXgnLh4+B5xQlNVz8Og8kvArMtNROxVQuCaSnIDdD5LKyWbRd2n9WGe2R8PzgCmr3EgVLrjyBxWezF0jLHwVN8efS3rCj/EWgvIWgb9tarpVUDK/b58Da+sqqls3eNbuv7pr+eoZG+SrDK6nWeL3c6H5Apxz7LjVc1uTIdsIXxuOLYA4/ilBmSVIzuDWfdRUfhHdY6+cn8HFRm+2hM8AnXGXws9555KrUB5qihylGa8subX2Nn6UwNR1AkUTV74bU='),
    # KSK-2010:
    dns.rrset.from_text('.', 15202, 'IN', 'DNSKEY', '257 3 8 AwEAAagAIKlVZrpC6Ia7gEzahOR+9W29euxhJhVVLOyQbSEW0O8gcCjF FVQUTf6v58fLjwBd0YI0EzrAcQqBGCzh/RStIoO8g0NfnfL2MTJRkxoX bfDaUeVPQuYEhg37NZWAJQ9VnMVDxP/VHL496M/QZxkjf5/Efucp2gaD X6RS6CXpoY68LsvPVjR0ZSwzz1apAzvN9dlzEheX7ICJBBtuA6G3LQpz W5hOA2hzCTMjJPJ8LbqF6dsV6DoBQzgul0sGIcGOYl7OyQdXfZ57relS Qageu+ipAdTTJ25AsRTAoub8ONGcLmqrAmRLKBP1dfwhYB4N7knNnulq QxA+Uk1ihz0='),
]

In [16]:
def get_anchor(year):
    ''' There are two anchors, get one of the anchors
    
    Args:
        year (int): 2017 or 2010
        
    Return:
        trusted root key singing key (str)
    '''
    if year == 2017:
        return trust_anchors[0].items[0].to_text()
    elif year == 2010:
        return trust_anchors[1].items[0].to_text()
    else:
        raise Exception('Parameter is neither 2017 nor 2010')

In [17]:
response2.answer[0]

<DNS . IN DNSKEY RRset>

In [18]:
# verify the root: compare the trust_anchors with the root's DNSKEY response

dnskeys, rrsig, name = get_rrset(response2, 'DNSKEY')
flag_trust = True
for dnskey in dnskeys:
    if dnskey.flags == 257:
        try:
            if dnskey.to_text() == get_anchor(2017):
                continue
            elif dnskey.to_text() == get_anchor(2010):
                continue
            else:
                flag_trust = False
                break
        except Exception as e:
            print(e)
            
print(flag_trust)

True


In [20]:
def verify_root(dnskey):
    '''Verify the root by comparing the pubksk in the response and the trusted pubksk
    
    Args:
        dnskey (dns.rrset.RRset)
        
    
    '''
    for dnskey in dnskeys:
        if dnskey.flags == 257:
            if dnskey.to_text() == get_anchor(2017):
                continue
            elif dnskey.to_text() == get_anchor(2010):
                continue
            else:
                raise Exception('Does not match trusted pubksk')
    else:
        print('Root verified')

In [21]:
type(dnskeys)

dns.rrset.RRset

In [22]:
response3 = dns.query.udp(a_query, '192.5.6.30')
print_response(response3)


 QUESTION:
paypal.com. IN A

 ANSWER:

 AUTHORITY:
paypal.com. 172800 IN NS pdns100.ultradns.net.
paypal.com. 172800 IN NS pdns100.ultradns.com.
paypal.com. 172800 IN NS ns1.p57.dynect.net.
paypal.com. 172800 IN NS ns2.p57.dynect.net.
paypal.com. 86400 IN DS 21037 5 2 0df17b28554954d819e0ceeab98fcfcd56572a4cf4f551f0a9be6d04db2f65c3
paypal.com. 86400 IN RRSIG DS 8 2 86400 20180225051533 20180218040533 46967 com. EbElQjsNEDtglmyE06dUwNhc3LPxQxNq NslvgrueZy3tMxQlTISyoEVEyrhtX7oV D1VbH5lbOq8IeNpw3sMMMaoxy4q6CBAl AwFYZCqak/bp/d//Yv138JWbiNClmCD/ 9yo11rITR1IHERqtxUQE8aIc8dXO9y8J rSww/oSuZwY=

 ADDITIONAL:
pdns100.ultradns.net. 172800 IN A 156.154.65.100
pdns100.ultradns.net. 172800 IN AAAA 2610:a1:1014::88
pdns100.ultradns.com. 172800 IN A 156.154.64.100
pdns100.ultradns.com. 172800 IN AAAA 2001:502:f3ff::88
ns1.p57.dynect.net. 172800 IN A 208.78.70.57
ns2.p57.dynect.net. 172800 IN A 204.13.250.57


In [23]:
c_query = dns.message.make_query('com.', 'DNSKEY', want_dnssec=True)
response4 = dns.query.udp(c_query, '192.5.6.30')
print_response(response4)


 QUESTION:
com. IN DNSKEY

 ANSWER:
com. 86400 IN DNSKEY 256 3 8 AQPLmJ3YIDDZPh9wYbc76pe1azHsCyIi DzBgf7fz+k4poMWHzE6f4My3VcPX2SIn /v0jPGBIEVZl2pPdY/IqOd5BfCsjrE+b BUYz6uxWiaT3wMuhfN1LNOazLQRlgU8R rEOgG1L4yFD8m9FVAisWsXLY+WQ8qIyr K/UIUwDmiFC5vQ==
com. 86400 IN DNSKEY 257 3 8 AQPDzldNmMvZFX4NcNJ0uEnKDg7tmv/F 3MyQR0lpBmVcNcsIszxNFxsBfKNW9JYC Yqpik8366LE7VbIcNRzfp2h9OO8HRl+H +E08zauK8k7evWEmu/6od+2boggPoiEf GNyvNPaSI7FOIroDsnw/taggzHRX1Z7S OiOiPWPNIwSUyWOZ79VmcQ1GLkC6NlYv G3HwYmynQv6oFwGv/KELSw7ZSdrbTQ0H XvZbqMUI7BaMskmvgm1G7oKZ1YiF7O9i oVNc0+7ASbqmZN7Z98EGU/Qh2K/BgUe8 Hs0XVcdPKrtyYnoQHd2ynKPcMMlTEih2 /2HDHjRPJ2aywIpKNnv4oPo/
com. 86400 IN RRSIG DNSKEY 8 1 86400 20180303192533 20180216192033 30909 com. KfTfHhzBbmAT4lLfrR8eSGeXr4QY068j 7gDmr7kW4vmSVoCPxBblazcCtT5Ew2um t0ZmyzkLzgxjjvtt7jgkML3z59HZRHm7 Y4y++xPcOQmK8CkeHhJnkoRdLeKxEZwS tlVBgML/udxBRlTp7c4YL7i62zfDNapa TqLWuXxsgOadOlcNgYRQH9qRWWc5578/ z7oP4NcHugOPCOgMPhXAJwn2Be5I9BZN SjeeJFiaG2xVuxuUqrArEdF6UZi2TB8+ U5Dcu3EGENIucovnt1O574azHc

In [24]:
# verify the DNSKEY rrset

try:
    dnskey, rrsig_key, name_key = get_rrset(response4, 'DNSKEY')
    dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DNSKEYs are good~')

Congrats! DNSKEYs are good~


In [25]:
# verify the DS rrset

try:
    ds, rrsig_ds, name_ds = get_rrset(response3, 'DS')
    dns.dnssec.validate(ds, rrsig_ds, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DS is good~')

Congrats! DS is good~


In [26]:
# verify the com. zone: do a hash on com's public key signing key, then see if equals to the DS in parent

rrset = response1.authority[1]
# rrset.rdtype == 43
trust_ds = rrset.items[0]
algorithm = 'SHA256' if trust_ds.digest_type == 2 else 'SHA1' 
name = dns.name.from_text('com.')
com_pubksk = response4.answer[0].items[1]
ds = dns.dnssec.make_ds(name, com_pubksk, algorithm)

if ds == trust_ds:
    print(True)
else:
    print(False)

True


In [33]:
# verify the com. zone: do a hash on com's public key signing key, then see if equals to the DS in parent

trust_ds, name = get_trust_ds(response1)
algorithm = 'SHA256' if trust_ds.digest_type == 2 else 'SHA1' 
pubksk = get_pubksk(response4)
ds = dns.dnssec.make_ds(name, pubksk, algorithm)

if ds == trust_ds:
    print(True)
else:
    print(False)

True


In [32]:
def verify_ds(response, name_key, dnskey):
    '''Verify the ds in the response.
    
    Args:
        response (dns.message.Message): a response that contains DS record
        name_key (dns.name.Name): name of zone that contains the DNSKEY
        dnskey (dns.rrset.RRset): rrset that contains public zone signing key
    '''
    try:
        ds, rrsig_ds, name_ds = get_rrset(response, 'DS')
        dns.dnssec.validate(ds, rrsig_ds, {name_key:dnskey})
    except Exception as e:
        raise e
    else:
        print('Congrats! DS is good~', name_ds)

In [31]:
def verify_zone(response, response_parent):
    '''Verify the zone: do a hash on the zone's public key signing key, then see if equals to the DS in parent
    
    Args:
        response (dns.message.Message): a response that contains pubksk to verify
        response_parent (dns.message.Message): a parent response that has the trusted DS
    '''
    trust_ds, name = get_trust_ds(response_parent)
    algorithm = 'SHA256' if trust_ds.digest_type ==2 else 'SHA1'
    pubksk = get_pubksk(response)
    ds = dns.dnssec.make_ds(name, pubksk, algorithm)
    
    if ds != trust_ds:
        raise Exception('DS does not match!', name)
        print('caitao')
    print('zhan')

In [ ]:
verify_zone(response4, response1)

In [30]:
def get_pubksk(response):
    '''Get public key signing key from response
    
    Args:
        response (dns.message.Message) that contains DNSKEY information
        
    Return:
        a (dns.rdtypes.ANY.DNSKEY.DNSKEY) that contains public key signing key
    '''
    dnskey, rrsig_key, name_key = get_rrset(response, 'DNSKEY')
    for item in dnskey:
        if item.flags == 257:
            return item

In [28]:
def get_trust_ds(response):
    '''Get trust ds digest from parent's response
    
    Args:
        response (dns.message.Message) from parent that contains child's DS information
        
    Return:
        trusted ds digest from parent (dns.rdtypes.ANY.DS.DS), and its name (dns.name.Name)
    '''
    ds, rrsig_ds, name_ds = get_rrset(response, 'DS')
    return ds.items[0], name_ds

In [ ]:
response5 = dns.query.udp(a_query, '156.154.65.100')
print_response(response5)

In [ ]:
d_query = dns.message.make_query('paypal.com.', 'DNSKEY', want_dnssec=True)
response6 = dns.query.udp(d_query, '156.154.65.100')
print_response(response6)

In [ ]:
# verify the DNSKEY rrset

try:
    dnskey, rrsig_key, name_key = get_rrset(response6, 'DNSKEY')
    dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DNSKEYs are good~')

In [ ]:
# verify the A rrset

try:
    a, rrsig_a, name_a = get_rrset(response5, 'A')
    dns.dnssec.validate(a, rrsig_a, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! A records are good~')

In [ ]:
# verify the NS rrset

try:
    ns, rrsig_ns, name_ns = get_rrset(response5, 'NS')
    dns.dnssec.validate(ns, rrsig_ns, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! NS records are good~')

In [ ]:
# verify the paypal.com. zone: do a hash on com's public key signing key, then see if equals to the DS in parent

trust_ds, name = get_trust_ds(response3)
algorithm = 'SHA256' if trust_ds.digest_type == 2 else 'SHA1' 
pubksk = get_pubksk(response6)
ds = dns.dnssec.make_ds(name, pubksk, algorithm)

if ds == trust_ds:
    print(True)
else:
    print(False)

# CNN

In [ ]:
a_query = dns.message.make_query('cnn.com', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print_response(response1)

In [ ]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print_response(response2)

In [ ]:
response3 = dns.query.udp(a_query, '192.5.6.30')
print_response(response3)

In [ ]:
def check_ds_exist(response):
    '''Check whether DS record exist in the response
    
    Args:
        response (dns.message.Message): a response
        
    Return:
        (bool) True or False
    '''
    flag = False
    for rrset in response.authority:
        if rrset.rdtype == rdtype_dic['DS']:
            flag = True
            break
    return flag

In [ ]:
check_ds_exist(response2)

In [ ]:
c_query = dns.message.make_query('com.', 'DNSKEY', want_dnssec=True)
response4 = dns.query.udp(c_query, '192.5.6.30')
print_response(response4)

In [ ]:
response6 = dns.query.udp(b_query, '205.251.192.47')
print_response(response6)

# dnssec-failed.org

In [ ]:
a_query = dns.message.make_query('dnssec-failed.org', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print_response(response1)

In [ ]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print_response(response2)

In [ ]:
name_key, dnskey = verify_dnskey(response2)
verify_ds(response1, name_key, dnskey)
verify_root(dnskey)

In [ ]:
response3 = dns.query.udp(a_query, '199.19.56.1')
print_response(response3)

In [47]:
c_query = dns.message.make_query('org.', 'RRSIG', want_dnssec=False)
response4 = dns.query.udp(c_query, '199.19.57.1')
print_response(response4)


 QUESTION:
org. IN RRSIG

 ANSWER:
org. 86400 IN RRSIG NS 7 1 86400 20180310152953 20180217142953 1862 org. GHoBxF9AC2tpfGa+R20XzlmFWOT/O+pi s3bH6Tb+Te6p15QfiGwv8IIYhiqH99IW qjvdA3YNnsPSuPaGWVtZ1qHxD9oTk2oo m+hZpSnbSGNCzPiq+T1a2RqGNwUwPYyn U2blrJftFuccDkyKEu8Rch1cbTL1e3qz EHS1gO0ryQI=
org. 900 IN RRSIG SOA 7 1 900 20180312132905 20180219122905 1862 org. TgvxKPfc6yecLYTuF1odi5XsNEMuD5Nn oMUHyKgini60QXX7gTY86lbLRWTvd9pH kzfFRX8IUfoSVkGZ3cYt3aDibJP6Lxfd VH+ojYjTHHASdTQdNdN/NvolIxMa7ppQ G4DYkuBefTnsdEyKFlbKk96DE5piuQ7m IL8p7Im0eHU=
org. 900 IN RRSIG NSEC3PARAM 7 1 900 20180310152953 20180217142953 1862 org. Mil6FzdZEV90g/vTIq6T+9zF8AWwlp06 uuoCSV9QOflabwZ5TrwrEQvzaw/oitXR SleAP4nSjVX1PN14z5SURCftu6XKwdJJ FKG+LIBbGYW+s2kMKh9UP4VNG46p8JA/ oLTtvF2tqAEN8h9gXCCJIJqQIZXOoziH oQhLiCF63Mg=

 AUTHORITY:

 ADDITIONAL:


In [ ]:
def verify_org_dnskey(ip):
    '''Zone org. has to query dnskey and rrsig separately. 
       There are 4 DNSKEYs for org, but you can not get all 4 of them in a single query.
       You have to query multiple times and union them until you get all the four. Shit!!!
    
    Args:
        ip (str): one ip of an org name server
        
    Return:
        (dnskey, name) if success
    '''
    dnskey_org = None
    name_org = None
    query_org = dns.message.make_query('org.', 'DNSKEY')
    response_org = dns.query.udp(query_org, ip)
    while len(response_org.answer) == 0:
        response_org = dns.query.udp(query_org, ip)
    dnskey_org = response_org.answer[0]
    name_org = dnskey_org.name
    while len(dnskey_org) != 4:
        query_org = dns.message.make_query('org.', 'DNSKEY')
        response_org = dns.query.udp(query_org, ip)
        try:
            dnskey_org.union_update(response_org.answer[0])
        except:
            pass
    
    rrsig_dnskey_org = None
    query_org = dns.message.make_query('org.', 'RRSIG')
    response_org = dns.query.udp(query_org, ip)
    while len(response_org.answer) == 0:
        response_org = dns.query.udp(query_org, ip)
    rrsig_dnskey_org = response_org.answer[0]
    
    try:
        dns.dnssec.validate(dnskey_org, rrsig_dnskey_org, {name_org:dnskey_org})
    except Exception as e:
        raise e
    else:
        print('Congrats! DNSKEYs are good~', name)
        return name_org, dnskey_org

In [ ]:
def verify_org_zone(dnskey_org, response_parent):
    '''Verify the zone: do a hash on the zone's public key signing key, then see if equals to the DS in parent
    
    Args:
        response (dns.message.Message): a response that contains pubksk to verify
        response_parent (dns.message.Message): a parent response that has the trusted DS
    '''
    trust_ds, name = get_trust_ds(response_parent)
    algorithm = 'SHA256' if trust_ds.digest_type ==2 else 'SHA1'
    
    dnskey_backup = dnskey_org
    
    for dnskey in dnskey_backup:
        if dnskey.flags == 256:
            dnskey_org.remove(dnskey)   # remove the public zone signing keys, so the public key signing keys are left
    
    ds2 = dns.dnssec.make_ds(name, dnskey_org.items[0], algorithm)
    ds1 = dns.dnssec.make_ds(name, dnskey_org.items[1], algorithm)
    if (ds1 == trust_ds or ds2 == trust_ds) == 0:
        raise Exception('DS does not match!', name)
    print('Congrads! Zone', name, 'verified')

In [ ]:
name_key, dnskey = verify_org_dnskey('199.19.57.1')
verify_ds(response3, name_key, dnskey)
verify_org_zone(dnskey, response1)

In [ ]:
response5 = dns.query.udp(a_query, '68.87.72.244')
print_response(response5)

In [ ]:
d_query = dns.message.make_query('dnssec-failed.org.', 'DNSKEY', want_dnssec=True)
response6 = dns.query.udp(d_query, '68.87.72.244')
print_response(response6)

In [ ]:
def verify_a(response, name_key, dnskey):
    '''Verify the A record in the response.
    
    Args:
        response (dns.message.Message): a response that contains A record
        name_key (dns.name.Name): name of zone that contains the DNSKEY
        dnskey (dns.rrset.RRset): rrset that contains public zone signing key
    '''
    try:
        a, rrsig_a, name_a = get_rrset(response, 'A')
        dns.dnssec.validate(a, rrsig_a, {name_key:dnskey})
    except Exception as e:
        raise e
    else:
        print('Congrats! A records are good~')

In [ ]:
name_key, dnskey = verify_dnskey(response6)
verify_a(response5, name_key, dnskey)
verify_zone(response6, response3)